In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt

This applies the process in Evaluate_crash_model_manual.ipynb (calculate crashes at each individual intersection, then find the sum) across all projects.

In [ ]:
crashes = pd.read_csv('output_2023_09_05/reports/safety-4-combined-b-crashes-all.csv')
segments = pd.read_csv('output_2023_09_05/reports/overall-5-ways.csv')
intersections = pd.read_csv('output_2023_09_05/reports/overall-6-intersections.csv')
alpha = pd.read_csv('output_2023_09_05/lookups/alpha.csv')
Ljvf = pd.read_csv('output_2023_09_05/reports/overall-3-reach-Ljvf.csv')
volume = pd.read_csv('output_2023_09_05/reports/safety-5-volume-d-combined.csv')
crash_model = pd.read_csv('output_2023_09_05/reports/safety-4-combined-a-crashes-model.csv')

In [ ]:
segments_n = segments.replace(["Not applicable","Not available"],np.NaN)
intersections_n = intersections.replace(["Not applicable","Not available"],np.NaN)

In [ ]:
segments_n["Bicycle exposure"]=pd.to_numeric(segments_n["Bicycle exposure"])
segments_n["Pedestrian exposure"]=pd.to_numeric(segments_n["Pedestrian exposure"])
intersections_n["Bicycle exposure"]=pd.to_numeric(intersections_n["Bicycle exposure"])
intersections_n["Pedestrian exposure"]=pd.to_numeric(intersections_n["Pedestrian exposure"])

In [ ]:
## Find e^alpha from alpha constant
alpha["e_alpha"] = np.exp(alpha["alpha"])

In [ ]:
## Rename columns to match each other and combine tables
segments_n["Bicycle volume class"] = segments_n["Bicycle volume class"].str.lower()
alpha_segment = alpha[alpha["location type"] == "roadway"].rename(columns={"volume":"Bicycle volume class","functional class":"Functional class"})
## eventually do this differently for each mode
segments_alpha = pd.merge(segments_n,alpha_segment, on=["Bicycle volume class","Functional class"])

## Convert to miles - what I didn't notice earlier!
segments_alpha["Length_miles"] = segments_alpha["Length"]/5280

In [ ]:
## Set volume separately by mode
segments_alpha.loc[segments_alpha["mode"] == "bicycling", "Volume"] = segments_alpha.loc[segments_alpha["mode"] == "bicycling", "Bicycle exposure"]
segments_alpha.loc[segments_alpha["mode"] == "walking", "Volume"] = segments_alpha.loc[segments_alpha["mode"] == "walking", "Pedestrian exposure"]
segments_alpha.loc[segments_alpha["mode"] == "combined", "Volume"] = (segments_alpha.loc[segments_alpha["mode"] == "combined", "Bicycle exposure"] + segments_alpha.loc[segments_alpha["mode"] == "combined", "Pedestrian exposure"])
segments_alpha["Volume_p"] = pow(segments_alpha["Volume"],0.5)

segments_alpha["e_alpha_Length"] = segments_alpha["e_alpha"]*segments_alpha["Length_miles"]
segments_alpha["e_alpha_Volume_p"] = segments_alpha["e_alpha"]*segments_alpha["Volume_p"]
segments_alpha["Crashes"] = segments_alpha["e_alpha_Length"]*segments_alpha["Volume_p"]

segments_alpha.groupby(["mode","outcome"])["Crashes"].plot(legend=True,figsize=(15,10))
## So this is the estimated crashes at each individual segment - which actually looks kind of reasonable. 0.1 - 0.2 crashes/year, totalled over all of the segments, "sounds ok"

In [ ]:
## Rename columns to match each other and combine tables
intersections_n["Pedestrian volume class"] = intersections_n["Pedestrian volume class"].str.lower()
alpha_intersection = alpha[alpha["location type"] == "intersection"].rename(columns={"volume":"Pedestrian volume class","functional class":"Functional class"})
intersections_alpha = pd.merge(intersections_n,alpha_intersection, on=["Pedestrian volume class","Functional class"])

In [ ]:
## Calculate volume separately by mode
intersections_alpha.loc[intersections_alpha["mode"] == "bicycling", "Volume"] = intersections_alpha.loc[intersections_alpha["mode"] == "bicycling", "Bicycle exposure"]
intersections_alpha.loc[intersections_alpha["mode"] == "walking", "Volume"] = intersections_alpha.loc[intersections_alpha["mode"] == "walking", "Pedestrian exposure"]
intersections_alpha.loc[intersections_alpha["mode"] == "combined", "Volume"] = (intersections_alpha.loc[intersections_alpha["mode"] == "combined", "Bicycle exposure"] + intersections_alpha.loc[intersections_alpha["mode"] == "combined", "Pedestrian exposure"])
intersections_alpha["Volume_p"] = pow(intersections_alpha["Volume"],0.5)

## No multiplying by length or count here - the e^alpha is just crashes/volume for that one intersection
intersections_alpha["e_alpha_Volume_p"] = intersections_alpha["e_alpha"]*intersections_alpha["Volume_p"]
intersections_alpha["Crashes"] = intersections_alpha["e_alpha_Volume_p"]
intersections_alpha.groupby(["mode","outcome"])["Crashes"].plot(legend=True,figsize=(15,10))
## Looks like bicycling is actually > than combined for most of them
# this is probably because the combined alpha constants are the average of bike and walk, so combined crashes is not actually the sum of bike crashes and walk crashes?

In [ ]:
## Total segment crashes
segments_alpha.groupby(["Project ID","mode","outcome"]).sum().groupby(["mode","outcome"])["Crashes"].plot()

In [ ]:
# compare to tool
crashes[crashes["J Location"] == "roadway"][crashes["K estimate"] == "mean"].groupby(["M Mode","O Outcome"])["ECmoj model"].plot(legend=True)

In [ ]:
## Total intersection crashes
intersections_alpha.groupby(["Project ID","mode","outcome"]).sum().groupby(["mode","outcome"])["Crashes"].plot(legend=True)

In [ ]:
# Compare to tool
crashes[crashes["J Location"] == "intersection"][crashes["K estimate"] == "mean"].groupby(["M Mode","O Outcome"])["ECmoj model"].plot(legend=True)

In [ ]:
crashes[crashes["J Location"] == "roadway"][crashes["K estimate"] == "mean"].groupby(["M Mode","O Outcome"])["ECmoj model"].plot(figsize=(15,10),legend=True)
segments_alpha.groupby(["Project ID","mode","outcome"]).sum().groupby(["mode","outcome"])["Crashes"].plot(figsize=(15,10),legend=True)

In [ ]:
intersections_alpha.groupby(["Project ID","mode","outcome"]).sum()["Crashes"].plot()

In [ ]:
crashes[crashes["J Location"] == "intersection"][crashes["K estimate"] == "mean"]["ECmoj model"].plot()

In [ ]:
.groupby(["M Mode","O Outcome"])
.groupby(["mode","outcome"])

["ECmoj model"].plot(figsize=(15,10),legend=True)

["Crashes"].plot(figsize=(15,10),legend=True)